In [9]:
from PIL import Image, ImageDraw
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import math
import random,evtk
import numpy as np

In [2]:
def sin(a):
    return math.sin(a)
def cos(a):
    return math.cos(a)
def tan(a):
    return math.tan(a)
pi=math.pi
sqrt2=math.sqrt(2)

def deg2rad(t):
    return t/180*pi
def rad2deg(t):
    return t/pi*180

def rotatePoint(p,c,t):
    '''
    rotate a point p=(px,py) about a point c=(cx,cy) by t radians counterclockwise
    '''
    px=p[0]; py=p[1]; px-=c[0]; py-=c[1]
    rx=px*cos(t)-py*sin(t)+c[0]
    ry=px*sin(t)+py*cos(t)+c[1]
    return (rx,ry)

def midpoint(a,b):
    return (a[0]/2+b[0]/2,a[1]/2+b[1]/2)

### Octagon Stuff

In [12]:
def OctPts(c, r, t):
    cx=c[0]; cy=c[1];
    points=[(cx,cy+r),(cx-r*sqrt2/2,cy+r*sqrt2/2),(cx-r,cy),(cx-r*sqrt2/2,cy-r*sqrt2/2),(cx,cy-r),(cx+r*sqrt2/2,cy-r*sqrt2/2),(cx+r,cy),(cx+r*sqrt2/2,cy+r*sqrt2/2)]
    for i in range(8): points[i]=rotatePoint(points[i],c,t)
    return points

def DrawBoard(c,r1,r2,t,w,draw):
    o1=OctPts(c,r1,t); draw.polygon(o1,outline='black',fill='white',width=w)
    o2=OctPts(c,r2,t); draw.polygon(o2,outline='black',fill='white',width=w)
    for i in range(8): draw.line([o1[i],o2[i]],fill='black',width=w)
    return o1, o2

### Chevron Stuff

In [24]:
def ChvPts(c,s,t):
    cx=c[0]; cy=c[1];
    points=[(cx,cy+s),(cx+s/2,cy+s/2),(cx+s/2,cy-s/2),(cx,cy),(cx-s/2,cy-s/2),(cx-s/2,cy+s/2)]
    for i in range(6): points[i]=rotatePoint(points[i],c,t)
    return points

def DrawChevron(c,s,t,draw,col):
    points = ChvPts(c,s,t)
    draw.polygon(points,fill=col)
    return points

### Generator

In [57]:

def Oct_vtk(o1, o2):
    xs = np.array([o[0] for o in o1] + [o[0] for o in o2])
    ys = np.array([o[1] for o in o1] + [o[1] for o in o2])
    zs = np.array([random.gauss(0, 0.01) for _ in xs])

    conn = [0]*32
    for i in range(8):
        conn[i*4] = i
        conn[i*4+1] = (i+1) % 8
        conn[i*4+2] = (i+1) % 8 + 8
        conn[i*4+3] = i+8

    conn += [i+8 for i in range(8)]
    conn = np.array(conn, dtype=int)
    offset = np.array([4*i for i in range(1,9)] + [40], dtype=int)

    ctype = np.array([evtk.vtk.VtkQuad.tid for _ in range(8)] + [evtk.vtk.VtkPolygon.tid], dtype=int)
    
    evtk.hl.unstructuredGridToVTK('board', xs, ys, zs, connectivity = conn, offsets = offset, cell_types = ctype)

def add_chevron_vtk(points, xs, ys, conn):
    xs += [p[0] for p in points]
    ys += [p[1] for p in points]
    if conn:
        conn += [conn[-1]+i for i in range(1,7)]
    else:
        conn += [i for i in range(0,6)]

def GenN(n,draw,mdraw,rigged=75):
    '''image assumes 500x500 image, rescale as needed'''
    ocx=random.randrange(100,400); ocy=random.randrange(100,400) #octagon center x, octagon center y
    ocr=random.randrange(100,300); orr=random.uniform(1.3,3) #octagon radius, octagon radius ratio
    ofa=random.uniform(0,2*pi); oow=random.randrange(2,8); #octagon angle, outline width
    csz=random.randrange(25,40); #chevron size

    o1, o2 = DrawBoard((ocx,ocy),ocr,ocr//orr,ofa,oow,draw)
    Oct_vtk(o1, o2)
    inOcto=Polygon(OctPts((ocx,ocy),ocr//orr,ofa))

    cspts=[]; cspolys=[]
    xs = []; ys=[]; conn = []

    for i in range(n):
        ccx=random.randrange(ocx-rigged,ocx+rigged); ccy=random.randrange(ocy-rigged,ocy+rigged); #chevron center x,y
        cfa=random.uniform(0,2*pi); #chevron angle
        points = DrawChevron((ccx,ccy),csz,cfa,draw,"red"); DrawChevron((ccx,ccy),csz,cfa,mdraw,"white")
        add_chevron_vtk(points, xs, ys, conn)
        cspts.append(ChvPts((ccx,ccy),csz,cfa)); cspolys.append(Polygon(ChvPts((ccx,ccy),csz,cfa))); 

    xs = np.array(xs)
    ys = np.array(ys)
    zs = np.array([random.gauss(0, 0.01) for _ in xs])
    conn = np.array(conn, dtype=int)
    offset = np.arange(6, n*6+1, 6)
    ctype = np.array([evtk.vtk.VtkPolygon.tid]*n, dtype=int)

    evtk.hl.unstructuredGridToVTK('chevrons', xs, ys, zs, connectivity = conn, offsets = offset, cell_types = ctype)

    allDistinct=True;
    for i in range(n-1):
        for j in range(i,n):
            for cpt in cspts[i]:
                if cspolys[j].contains(Point(cpt[0],cpt[1])):allDistinct=False; break;
    return allDistinct

def Gen4(i):
    ad=False
    im=Image.open('bg/bluegradient500x500.jpeg');
    mim=Image.open('bg/pureblack500x500.png');
    while (ad==False):
        im=Image.open('bg/bluegradient500x500.jpeg'); draw=ImageDraw.Draw(im)
        mim=Image.open('bg/pureblack500x500.png'); mdraw=ImageDraw.Draw(mim)
        ad=GenN(4,draw,mdraw) 
    id='0'*(5-len(str(i)))+str(i)
    im.save('IMG'+id+'.jpeg','JPEG')
    mim.save('MSK'+id+'.jpeg','JPEG')

In [58]:
im=Image.open('bluegradient500x500.jpeg'); draw=ImageDraw.Draw(im)
mim=Image.open('pureblack500x500.png'); mdraw=ImageDraw.Draw(mim)
ad=GenN(4,draw,mdraw)